# GPU Acceleration Basics 03
This notebook follows the video Python CUDA Installation & CUPY | GPU Acceleration Basics 03 by Rounak Paul found on YouTube.

## GPU Memory
The code below demonstrates the use of CUDA with Numba to do matrix multiplication using both global and shared memory. The first part shows a simple element-wise addition of two arrays, while the second part demonstrates a more complex matrix multiplication using shared memory for optimization. The results are verified against Python's built-in matrix multiplication to ensure correctness. The use of shared memory can significantly improve performance for large matrices  reduces the number of global memory accesses and allows for faster data sharing between threads within a block.

In [20]:
import numpy as np
from numba import cuda, float32
import math

In [21]:
@cuda.jit
def foo(a, b, c):
    pos = cuda.grid(1)
    size =  len(c)

    if pos < size:
        c[pos] = a[pos] + b[pos]


In [22]:
N = 100000
a = cuda.to_device(np.random.random(N))
b = cuda.to_device(np.random.random(N))
c = cuda.device_array_like(a)

This function, 'forall()' creas a 1D grid for a given data size

In [23]:
foo.forall(len(a))(a, b, c)
c.copy_to_host()

array([1.04683472, 0.6329102 , 0.26522472, ..., 0.30387198, 0.83543071,
       1.21570496], shape=(100000,))

In [24]:
nthreads = 256
nblocks = (len(a) // nthreads) + 1
foo[nblocks, nthreads](a, b, c)
c.copy_to_host()

array([1.04683472, 0.6329102 , 0.26522472, ..., 0.30387198, 0.83543071,
       1.21570496], shape=(100000,))

## Matrix Multiplication

In [25]:
@cuda.jit
def matrix_mult(A, B, C):
    # Perform square matrix multiplication C = A * B
    i, j = cuda.grid(2)

    # This condition is necessary to avoid accessing out of bounds
    if i < C.shape[0] and j < C.shape[1]:
        for k in range(A.shape[1]):
            C[i, j] += A[i, k] * B[k, j]


In [26]:
x_h = np.arange(16).reshape([4, 4])
y_h = np.ones([4, 4])
z_h = np.zeros([4, 4])

x_d = cuda.to_device(x_h)
y_d = cuda.to_device(y_h)
z_d = cuda.to_device(z_h)

In [27]:
tpb = (16, 16)
bpg_x = math.ceil(z_h.shape[0] / tpb[0])
bpg_y = math.ceil(z_h.shape[1] / tpb[1])
bpg = (bpg_x, bpg_y)

In [28]:
matrix_mult[bpg, tpb](x_d, y_d, z_d)

/home/eddie-water/dev/python/sub-shader/env/lib/python3.12/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [29]:
z_h = z_d.copy_to_host()
print("Result of Z = X * Y\n", z_h)

Result of Z = X * Y
 [[ 6.  6.  6.  6.]
 [22. 22. 22. 22.]
 [38. 38. 38. 38.]
 [54. 54. 54. 54.]]


## Matrix Multiplication - Improved with Shared Memory

In [30]:
TPB = 16

@cuda.jit
def fast_matrix_mult(A, B, C):
    # Define an array in shared memory
    sA = cuda.shared.array((TPB, TPB), dtype=float32)
    sB = cuda.shared.array((TPB, TPB), dtype=float32)

    # Get the row and column index of the C element
    x, y = cuda.grid(2)
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x
    
    temp = float32(0.0)

    for i in range(bpg):
        # Preload data into shared memory
        sA[tx, ty] = 0
        sB[tx, ty] = 0
        if (x < A.shape[0]) and ((tx + i * TPB) < A.shape[1]):
            sA[ty, tx] = A[y, tx + i * TPB]
        if (y < B.shape[1]) and ((ty + i * TPB) < B.shape[0]):
            sB[ty, tx] = B[ty + i * TPB , x]

        # Synchronize to ensure all threads have loaded the data
        cuda.syncthreads()

        # Perform the multiplication
        for j in range(TPB):
            temp += sA[ty, j] * sB[j, tx]

        # Synchronize to ensure all threads have completed the mults
        cuda.syncthreads()
    
    if y < C.shape[0] and x < C.shape[1]:
        C[y, x] = temp

In [31]:
# Reinitialize the matrices
x_h = np.arange(16).reshape([4, 4])
y_h = np.ones([4, 4])
z_h = np.zeros([4, 4])

x_d = cuda.to_device(x_h)
y_d = cuda.to_device(y_h)
z_d = cuda.to_device(z_h)

In [32]:
tpb = (TPB, TPB)
bpg_x = math.ceil(z_h.shape[0] / tpb[0])
bpg_y = math.ceil(z_h.shape[1] / tpb[1])
bpg = (bpg_x, bpg_y)

In [33]:
fast_matrix_mult[bpg, tpb](x_d, y_d, z_d)
z_h = z_d.copy_to_host()

print("Result of Z = X * Y with shared memory\n", z_h)
print()
print("Same result as Python's built-in matrix multiplication? :", np.allclose(z_h, x_h @ y_h))

Result of Z = X * Y with shared memory
 [[ 6.  6.  6.  6.]
 [22. 22. 22. 22.]
 [38. 38. 38. 38.]
 [54. 54. 54. 54.]]

Same result as Python's built-in matrix multiplication? : True


/home/eddie-water/dev/python/sub-shader/env/lib/python3.12/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


# Compare the Two

In [34]:
# Reinitialize the matrices
x_h = np.arange(16).reshape([4, 4])
y_h = np.ones([4, 4])
z_h = np.zeros([4, 4])

x_d = cuda.to_device(x_h)
y_d = cuda.to_device(y_h)
z_d = cuda.to_device(z_h)

t_original = %timeit -o matrix_mult[bpg, tpb](x_d, y_d, z_d)

# Reinitialize the matrices
x_h = np.arange(16).reshape([4, 4])
y_h = np.ones([4, 4])
z_h = np.zeros([4, 4])

x_d = cuda.to_device(x_h)
y_d = cuda.to_device(y_h)
z_d = cuda.to_device(z_h)
t_shared_mem = %timeit -o fast_matrix_mult[bpg, tpb](x_d, y_d, z_d)

t_ratio_avg = t_original.average / t_original.average

print()
print(f"The Shared Memory method is on average {t_ratio_avg:.2f} times faster")

19.9 μs ± 3.54 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
20.5 μs ± 568 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)

The Shared Memory method is on average 1.00 times faster


## Conclusions
Well it looks like we got worse performance. So cool!